In [2]:
from azureml.core import Workspace, Experiment

Workspace.
ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-234286")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(a9aa6499-d59e-40b5-b38a-66c3580a12fa, 1, 14, Finished, Available)

SyntaxError: invalid syntax (723965264.py, line 3)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project1_cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azure.ai.ml.entities import AmlCompute

cluster_basic = AmlCompute(
    name="basic-example",
    type="amlcompute",
    size="STANDARD_DS2_v2",
    location="westus",
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster_basic).result()
### YOUR CODE HERE ###

StatementMeta(, , , Cancelled, )

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':(0.1,1.0) , '--max_iter':choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15,delay_evaluation=5)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig("./", "train.py")### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy=policy,
                                    run_config=src,
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###


print("best run details :", hyperdrive_best_run.get_details())
print('\n')
print("best run file names :", hyperdrive_best_run.get_file_names())
print('\n')
print("best run metrics :", hyperdrive_best_run.get_metrics())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy" : hyperdrive_best_run.get_metrics()["Accuracy"]})

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tbds = TabularDatasetFactory(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(tbds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run
automl_run = exp.submit(config=automl_config, show_output=True)

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
remote_run.get_metrics()

from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './/best_model.onnx' )
